In [ ]:
# https://github.com/django/django/blob/c8eb9a7c451f7935a9eaafbb195acf2aa9fa867d/django/views/generic/list.py#L150



"""
- class ListView(MultipleObjectTemplateResponseMixin, BaseListView): - потомок двух классов
- class BaseListView(MultipleObjectMixin, View): - потомок еще двух классов
- class MultipleObjectTemplateResponseMixin(TemplateResponseMixin): - потомок еще одного родителя
- 
- вот та иерархия, которой не хватало прошлый раз

- попробую разобрать цепочку и добраться до истины) познать механизм.. начинаю с базового класса

"""

In [ ]:
# 1. Разберем ближний класс родитель, который также наследуется от двух других классов. )  "создаёт список чего-либо"

class BaseListView(MultipleObjectMixin, View):       # и этот класс в свою очередь имеет два родителя, один из которых я разбирал прошлый раз
    """Базовый вид для отображения списка объектов."""

    def get(self, request, *args, **kwargs):         # он вызывается когда делается HTTP запрос GET
        self.object_list = self.get_queryset()       # сохраняем в object_list, количество страниц - то что я разбирал в прошлом уроке
        allow_empty = self.get_allow_empty()         # метод вкладывается в переменную, которая отвечат за ошибку, если False, будет 404 ошибка
                                                     # 
        if not allow_empty:                          # а если он не False
            # Когда разбиение на страницы включено и object_list является набором запросов,
            # лучше сделать запрос, чем загружать неразбитый
            # набор запросов в памяти (перевод описания)
            if self.get_paginate_by(self.object_list) is not None and hasattr(           # если для списка обьектов включена пагинация страниц
                self.object_list, "exists"           #  результатом является True, если строка является именем одного из атрибутов объекта, и False, если нет
            ):
                is_empty = not self.object_list.exists()   # итак is_empty если False               
            else:
                is_empty = not self.object_list
            if is_empty:                                   # иначе вызвать 404
                raise Http404(
                    _("Empty list and “%(class_name)s.allow_empty” is False.")
                    % {
                        "class_name": self.__class__.__name__,
                    }
                )
        context = self.get_context_data()                # данные которые должен отобразить в шаблоне
        return self.render_to_response(context)          # используется для возврата HTTP ответа
    
    
# я конечно понимаю, что разработчики которые писали этот код хотели использовать один и тот же код, или меньше строк кода, но так или иначе есть что есть
# то есть сам ListView расширяется методами и атрибутами классов родителей, и уже в документации джанго ты играешь по этим правилам) 
# где то берут у родителя, данные берут из БД или используеют значения из методов, что то проверяют, используют, переиспользуют и имеют в итоге класс на выходе с некими возможностями

In [6]:
# проверка функции hasattr
class MyClass:
    def __init__(self, atr1, atr2):
        self.atr1 = atr1
        self.atr2 = atr2

my_obj = MyClass("Hi", "Fi")

print(hasattr(my_obj, "atr1")) # типа экземпляр имеет такой атрибут да или нет
print(hasattr(my_obj, "atr3")) 
print(hasattr(my_obj, "Hi")) 
print(hasattr(my_obj, "Fi")) 

True
False
False
False


In [ ]:
# тут нужно разобрать View, еще одного родителя BaseListView - так как он огромный, разберу в отдельном файле

In [ ]:
# 3. Третий на очереди class MultipleObjectTemplateResponseMixin(TemplateResponseMixin):

class MultipleObjectTemplateResponseMixin(TemplateResponseMixin):
    """ Класс примеси, который выполняет рендеринг ответа на основе шаблона для представлений, которые работают со списком экземпляров объекта"""

    template_name_suffix = "_list"                       # собственный атрибут, будет использоваться как добавление строки

    def get_template_names(self):
        """
        Возвращает список имен шаблонов, которые будут использоваться для запроса. 
        Должен вернуться список. Не может вызываться, если параметр render_to_response переопределен.
        
        """
        try:
            names = super().get_template_names()    # а если нам передано имя, то дальше код даже не выполняется
        except ImproperlyConfigured:                # исключение возникает, когда Django неправильно настроен
            # Если имя_шаблона не указано, это не проблема --
             # мы просто начинаем с пустого списка.
            names = []

         # Если список представляет собой набор запросов, мы придумаем имя шаблона на основе
         # приложения и названия модели. Это имя помещается в конец шаблона.
         # список имен, чтобы введенные пользователем имена переопределяли автоматически созданных ( перевод описания)
        if hasattr(self.object_list, "model"):       # ну типа если есть в нашем списке модель, то
            opts = self.object_list.model._meta      # присваиваем ._meta
            names.append(                            # согласно верхнему 
                "%s/%s%s.html"
                % (opts.app_label, opts.model_name, self.template_name_suffix)
            )
        elif not names:                              # короче этот блок кода автоматом присваивает автоматически имя, имя модели, формат и расширение
            raise ImproperlyConfigured(
                "%(cls)s requires either a 'template_name' attribute "
                "or a get_queryset() method that returns a QuerySet."
                % {
                    "cls": self.__class__.__name__,
                }
            )
        return names                                 # и вот тут возвращается уже автоматически созданное имя
# в общем потихоньку, по строчке, по букве собираю общую картину, пытаюсь понять общую реализацию класса ListView
# есть непонимание некоротых вещей, думаю чуть позже найду ответы, когда доберусь к самому главному родителю

In [ ]:
# 4. Четвёртый на очереди class TemplateResponseMixin:
class TemplateResponseMixin:               # этот класс имееет всего два метода и ни от кого не наследуется
    """ Миксин, который можно использовать для рендеринга шаблона """

    template_name = None            # этот атртибут я знаю из джанго.Views, мы через него пробрасываем наш шаблон
    template_engine = None
    response_class = TemplateResponse    # этот атрибут и есть присвоенный классу реализованный будет ниже
    content_type = None

    def render_to_response(self, context, **response_kwargs):           # передается self, переменная context и response_kwargs передаётся как словарь 
        """
        Возвращает ответ, используя `response_class` для этого представления, 
        с шаблоном, отображаемым с заданным контекстом.
        Передайте response_kwargs конструктору класса ответа.
        (описание)
        
        """
        response_kwargs.setdefault("content_type", self.content_type)     # этот метод словаря 
        return self.response_class(
            request=self.request,                 # возвращает данный обьект
            template=self.get_template_names(),   # возвращает шаблон с помощью метода
            context=context,                      # возвращает значение переменной (получается снова что этот метод только создаётся снизу, а тут уе возвращается)
            using=self.template_engine,           # какой то механизм шаблона для джанго
            **response_kwargs,                    # ну и вернуть сам словарь
        )

    def get_template_names(self):
        """
        Возвращает список имен шаблонов, которые будут использоваться для запроса. Должен вернуться
        список. Не может вызываться, если функция render_to_response() переопределена.
        
        """
        if self.template_name is None:
            raise ImproperlyConfigured(
                 "TemplateResponseMixin требует определения "
                 "имя_шаблона" или реализация 'get_template_names()'"
            )
        else:
            return [self.template_name]       # если шаблон не определен, то вызвать исключение или вернуть если есть

In [16]:
dict = {'Name': 'Zara', 'Age': 7}                 # set_default -- метод говорит сам за себя и устанавливает default ключом, если он не доступен
print ("Value : %s" %  dict.setdefault('Age', None))
print ("Value : %s" %  dict.setdefault('Sex', None))
print (dict)


Value : 7
Value : None
{'Name': 'Zara', 'Age': 7, 'Sex': None}


In [ ]:
# 5. class TemplateResponse(SimpleTemplateResponse): - пятый класс реализованный по цепочке для атрибута response_class
class TemplateResponse(SimpleTemplateResponse):  # ну конечно же унаследован еще от одного класса
    rendering_attrs = SimpleTemplateResponse.rendering_attrs + ["_request"]      # тут идёт вообще добавление атрибута с изменением

    def __init__(                      # конструтктор принимает следующие аргументы
        self,
        request,                       # экземпляр HttpRequest
        template,                      # шаблон
        context=None,                  # контекст
        content_type=None,             # тип он же формат контекста
        status=None,                   # состояние HTTP код ответа
        charset=None,                  # кодировка
        using=None,                    # строка, представляющая механизм шаблонов
        headers=None,                  # заголовки
    ):
        super().__init__(              # этот метод показывает явное наследование
            template, context, content_type, status, charset, using, headers=headers
        )
        self._request = request

In [ ]:
# 6. class SimpleTemplateResponse(HttpResponse): - ну и вот сам родитель https://github.com/django/django/blob/c8eb9a7c451f7935a9eaafbb195acf2aa9fa867d/django/template/response.py#L10